In [ ]:
import pandas as pd

path = r"C:\Users\Meus Documentos\Downloads\SIPRI-Milex-data-1949-2024_2.xlsx"

current = pd.read_excel(path, sheet_name='Current US$')
gdp_share = pd.read_excel(path, sheet_name='Share of GDP')
gov_share = pd.read_excel(path, sheet_name='Share of Govt. spending')
per_capita = pd.read_excel(path, sheet_name='Per capita')

print(current.head())


